In [ ]:
import os
import xarray as xr
from tqdm import tqdm
from wrf import getvar
import concurrent.futures
from netCDF4 import Dataset

In [ ]:
# Create a list of all WRF file directories
case_study_name =  'case_study_5'
save_folder = 'theta_cape_5'

data_dir = '/scratch3/thomasl/work/data/{}/'.format(case_study_name)

data_path= '/scratch3/thomasl/work/data/'

wrflist = list()
for (dirpath, dirnames, filenames) in os.walk(data_dir):
    wrflist += [ os.path.join(dirpath, file) for file in filenames ]

In [ ]:
# Add the variable to the wrf files with multiprocessing

def any_function(id):
    data_path= '/scratch3/thomasl/work/data/'
    
    ncfile = Dataset(wrflist[id])
    ds = xr.open_dataset(wrflist[id])

    theta_e = getvar(ncfile, 'theta_e')
    ds['theta_e'] = (['bottom_top', 'south_north', 'west_east'], theta_e)
    ds["theta_e"].attrs['units'] = 'K'
    ds["theta_e"].attrs['description'] = 'Equivalent Potential Temperture (K)'

    ds_theta_cape = ds[['theta_e', 'U', 'V', 'W', 'XLONG', 'XLAT', 'PB', 'P', 'PH', 'PHB', 'HGT', 'COSALPHA', 'SINALPHA']]
    ds_theta_cape.to_netcdf('{}{}/{}.nc'.format(data_path, save_folder, wrflist[id][-30:]),
                        format='NETCDF3_64BIT', mode='w')
    return id
    
def run(f, myIter):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(f, myIter), total = len(myIter)))
    return results

results = run(any_function, range(len(wrflist)))

result_as_list = list(results)